In [10]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
#导入langgraph检查点，用于持久化状态
from langgraph.checkpoint.memory import MemorySaver
#导入状态图和状态
from langgraph.graph import END, StateGraph, MessagesState
#导入工具节点（工具节点：用于调用工具）
from langgraph.prebuilt import ToolNode

定义工具函数，用于代理调用外部工具

In [11]:
@tool
def search(query:str):
    "模拟一个搜索工具"
    query_lower = query.lower()
    if "北京" in query_lower or "beijing" in query_lower:
        return "天气很好，适合外出旅行！"
    return "预计有大雨，建议避免外出。"


In [12]:
#将工具函数放入工具列表
tools = [search]

In [13]:
#创建工具节点
tool_node = ToolNode(tools)

In [ ]:
#初始化模型和工具，定义模型，并绑定工具到模型
#大模型根据任务和工具的匹配情况决定是否调用工具
api_key = ""
base_url = ""
model = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 0,openai_api_key=api_key,base_url=base_url )

In [15]:
def should_continue(state: MessagesState) -> Literal["tools",END]:
    message = state["messages"]
    last_message = message[-1]
    #如果LLM调用了工具，跳转到tools节点
    if last_message.tool_calls:
        return "tools"
    #否则，回复用户
    return END
#state是一个包含对话信息的状态对象

In [16]:
#定义调用大模型函数
def call_model(state: MessagesState):
    messages = state["messages"]
    model_with_tools = model.bind_tools(tools, tool_choice="auto", )
    response = model_with_tools.invoke(messages)
    print(response.tool_calls)
    return {"messages": [response]}

In [17]:
#创建状态传输图
workflow = StateGraph(MessagesState)

workflow.add_node("节点名称", 节点函数)<br>
执行节点时会自动调用节点函数

In [18]:
#定义图节点
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [19]:
#设置入口节点为agent
workflow.set_entry_point("agent")

In [20]:
#设置条件边
#条件边是在调用agent节点后再采取的
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

普通边：workflow.add_edge("起始节点","结束节点")

In [21]:
workflow.add_edge("tools","agent")

In [22]:
#初始化存储持久化状态
#也可以存再redis，MongoDB
checkpointer = MemorySaver()

编译图<br>
将其编译成一个LangChain可运行对象<br>
可以起到预检查的作用

In [23]:
app = workflow.compile(checkpointer=checkpointer)

执行图，调用可运行对象

In [24]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="北京今天的天气如何？")]},
    config = {"configurable": {"thread_id": 42}}
)

result = final_state["messages"][-1].content
print(result)

final_state = app.invoke(
    {"messages": [HumanMessage(content="我问的哪个城市？")]},
    config = {"configurable": {"thread_id": 42}}
)
result = final_state["messages"][-1].content
print(result)

[{'name': 'search', 'args': {'query': '北京今天的天气'}, 'id': 'call_7ZUdLlFYvHr1ZfCZ99SV6TX0', 'type': 'tool_call'}]
[]
天气很好，适合外出旅行！
[]
您问的是北京的天气情况。


In [25]:
graph_png = app.get_graph().draw_mermaid_png()
with open("langgraph_base.png","wb") as f:
    f.write(graph_png)

In [ ]:
#检测API是否可用
from openai import OpenAI
import os

# 明确设置环境变量
os.environ['OPENAI_BASE_URL'] = ""
os.environ['OPENAI_API_KEY'] = ""

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  # 添加 api_key 参数
    base_url=os.getenv("OPENAI_BASE_URL")
)

# 测试 API 是否连接成功
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Say hello to the world"}
        ]
    )
    print("API连接成功，响应：", response)
except Exception as e:
    print("API连接失败，错误信息：", e)


API连接成功，响应： <!doctype html><html lang="zh-CN"><head><meta charset="utf-8"/><link rel="icon" href="logo.png"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#ffffff"/><meta name="description" content="提供稳定可靠的Openai API转发公益免费的GPT-3.5 API服务。"/><title>FREE GPT API</title><script defer="defer" src="/static/js/main.53db0764.js"></script><link href="/static/css/main.e5f231c8.css" rel="stylesheet"></head><body><div id="root"></div></body></html>


调了半天大模型一直不按我预设的输出，仔细检查发现大模型调用了我的tools但觉得这个不合理所以又否掉了<br>
根据问题写了一个更合适的答案于是就被大模型认可并输出了，未曾设想的解决方法。